In [1]:
import pandas as pd
from scipy.interpolate import interp1d
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import price_data as price
import statistics
import time
from datetime import date,datetime
import os
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport
import chart

# Select your transport with a defined url endpoint
transport = AIOHTTPTransport(url="https://saturn.hasura.app/v1/graphql", headers={'x-hasura-admin-secret': 'Rc07SJt4ryC6RyNXDKFRAtFmRkGBbT8Ez3SdaEYsHQoHemCldvs52Kc803oK8X62'})

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

async def get_stored_data(symbol,timeframe):
    # Provide a GraphQL query
    split_symbol=symbol.split('/')
    base_currency=split_symbol[0]
    quote_currency=split_symbol[1]
    table=base_currency+quote_currency+'_'+timeframe
    if table=='BTCUSD_1d':
        query = gql(
            """
            query MyQuery {
                BTCUSD_1d {
                    unix
                    close
                    high
                    low
                    open
                }
            }
        """
        )
    elif table=='ETHUSD_1d':
        query = gql(
            """
            query MyQuery {
                ETHUSD_1d {
                    unix
                    close
                    high
                    low
                    open
                }
            }
        """
        )
    elif table=='ETHBTC_1d':
        query = gql(
            """
            query MyQuery {
                ETHBTC_1d {
                    unix
                    close
                    high
                    low
                    open
                }
            }
        """
        )
    else:
        return 'no such table'

    # Execute the query on the transport
    result = await client.execute_async(query)
    candles=result[table]
    df=pd.DataFrame({},columns=['unix','close','high','low','open'])
    for candle in candles:
        df=df.append(candle,ignore_index=True)

    return df.sort_values(by=['unix'], ignore_index=True)

    return result

def find_start(timestamps):
    start_found=False
    timestamps=list(map(lambda x:x[0]/1000,timestamps))
    index=len(timestamps)-1
    while not(start_found):
        print(timestamps[index])
        day=date.fromtimestamp(timestamps[index]).weekday()
        if day==0:
            start_found=True
        else:   
            index=index-1
    return index

def read_data(filename):
    raw=pd.read_csv('data/'+filename)
    timestamps=raw['unix']
    for i in range(len(timestamps)):
        if np.log10(timestamps.iloc[i])<12:
            raw.loc[i,'unix']=timestamps[i]*1000
    return raw.sort_values(by=['unix'], ignore_index=True)

# Creating Heikin Ashi candles
First we must download the candlestick data

In [27]:
eth=await get_stored_data('ETH/USD','1d')
weekly_candles=price.get_price_data('1w',data=eth)
weekly_candles

,unix,open,high,low,close
0,1.458432e+12,15.07,15.07,8.338,10.180
1,1.459037e+12,10.18,12.47,10.140,10.400
2,1.459642e+12,10.40,12.30,10.400,11.750
3,1.460246e+12,11.60,11.67,7.520,8.757
4,1.460851e+12,8.56,10.90,6.855,9.455
...,...,...,...,...,...
302,1.641082e+12,4066.20,4128.90,3582.600,3829.200
303,1.641686e+12,3829.20,3896.00,2998.000,3150.800
304,1.642291e+12,3150.80,3413.70,2921.800,3348.300
305,1.642896e+12,3348.30,3359.20,2300.000,2540.500


In [6]:
weekly_candles.iloc[-1][["open","high","low","close"]].values

array([3348.3, 3359.2, 2463.4, 2600.3])

Then we define a function to convert each candle stick into Heikin Ashi Candlestick

In [37]:
def heikin_ashi(previous_open, previous_close, candle):
    price_values = candle[["open","high","low","close"]]
    print(price_values)
    close = np.mean(price_values)
    open_price = 0.5*(previous_open+previous_close)
    high=max(price_values)
    low = min(price_values)

    return  candle["unix"], open_price, high, close, low

def convert_data_to_heikin_ashi(data):
    timestamps, opens, closes, highs, lows  = [0], [0],[0],[0],[0]
    #initialise values
    for i in range(1,len(data)):
        timestamp, open_price, high, close, low = heikin_ashi(opens[-1], closes[-1],data.iloc[i])
        timestamps.append(timestamp)
        opens.append(open_price)
        highs.append(high)
        lows.append(low)
        closes.append(close)

    return pd.DataFrame({'unix':timestamps,'open':opens,'high':highs,'low':lows,'close':closes}).sort_values(by=['unix'], ignore_index=True)


Then we convert weekly candles and observe

In [38]:
convert_data_to_heikin_ashi(weekly_candles)

open     10.18
high     12.47
low      10.14
close    10.40
Name: 1, dtype: float64
open     10.40
high     12.30
low      10.40
close    11.75
Name: 2, dtype: float64
open     11.600
high     11.670
low       7.520
close     8.757
Name: 3, dtype: float64
open      8.560
high     10.900
low       6.855
close     9.455
Name: 4, dtype: float64
open     9.302
high     9.550
low      7.730
close    8.097
Name: 5, dtype: float64
open     8.050
high     9.360
low      7.120
close    8.752
Name: 6, dtype: float64
open      8.828
high     10.300
low       8.789
close     9.500
Name: 7, dtype: float64
open      9.510
high     11.110
low       9.252
close     9.966
Name: 8, dtype: float64
open     10.01
high     14.89
low      10.00
close    14.21
Name: 9, dtype: float64
open     14.21
high     14.49
low      10.38
close    12.41
Name: 10, dtype: float64
open     12.44
high     14.30
low      12.30
close    13.95
Name: 11, dtype: float64
open     14.00
high     16.09
low      13.56
close    15.8

,unix,open,high,low,close
0,0.000000e+00,0.000000,0.00,0.000,0.00000
1,1.459037e+12,0.000000,12.47,10.140,10.79750
2,1.459642e+12,5.398750,12.30,10.400,11.21250
3,1.460246e+12,8.305625,11.67,7.520,9.88675
4,1.460851e+12,9.096187,10.90,6.855,8.94250
...,...,...,...,...,...
302,1.641082e+12,4031.798491,4128.90,3582.600,3901.72500
303,1.641686e+12,3966.761745,3896.00,2998.000,3468.50000
304,1.642291e+12,3717.630873,3413.70,2921.800,3208.65000
305,1.642896e+12,3463.140436,3359.20,2300.000,2887.00000


In [26]:
weekly_candles

,unix,open,high,low,close
0,1.458432e+12,15.07,15.07,8.338,10.180
1,1.459037e+12,10.18,12.47,10.140,10.400
2,1.459642e+12,10.40,12.30,10.400,11.750
3,1.460246e+12,11.60,11.67,7.520,8.757
4,1.460851e+12,8.56,10.90,6.855,9.455
...,...,...,...,...,...
301,1.640477e+12,3925.40,4160.00,3752.000,4066.200
302,1.641082e+12,4066.20,4128.90,3582.600,3829.200
303,1.641686e+12,3829.20,3896.00,2998.000,3150.800
304,1.642291e+12,3150.80,3413.70,2921.800,3348.300
